In [75]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

# 预处理

In [76]:
data_pd = pd.read_csv('./../')
print(data_pd)

                                                    file  \
0      CWE_79__POST__func_FILTER-CLEANING-email_filte...   
1      CWE_79__system__func_urlencode__Use_untrusted_...   
2      CWE_79__object-directGet__no_sanitizing__Unsaf...   
3      CWE_79__proc_open__func_FILTER-VALIDATION-numb...   
4      CWE_79__object-Array__ternary_white_list__Use_...   
...                                                  ...   
10075  CWE_79__object-indexArray__func_urlencode__Use...   
10076  CWE_79__backticks__CAST-cast_float_sort_of__Us...   
10077  CWE_79__POST__CAST-cast_float__Use_untrusted_d...   
10078  CWE_79__object-classicGet__CAST-func_settype_i...   
10079  CWE_79__popen__func_FILTER-CLEANING-full_speci...   

                                              sourcecode  \
0      <!-- \nUnsafe sample\ninput : get the field Us...   
1      <!-- \nUnsafe sample\ninput : execute a ls com...   
2      <!-- \nUnsafe sample\ninput : get the field us...   
3      <!-- \nUnsafe sample\ninput : us

In [77]:
code_list = data_pd['sourcecode']
code_list = np.array(code_list)
print(code_list.shape)

(10080,)


In [91]:
ptrn = r'echo\s(.+)\s*;'
# ptrn2 = r'echo\s\"(.+)\"\s*'

sink_list = []
for code in code_list:
    sink = re.findall(pattern=ptrn, string=code)
    if sink == []:
        print(code)
    if type(sink) == 'str':
        sink_list.append(sink)
    else:
        for element in sink:
            sink_list.append(element)
#     print(type(sink))
sink_list = np.array(sink_list)
print(sink_list.shape)
print(sink_list)

(10080,)
['$tainted ' '"body { color :\\"". $tainted ."\\" ; }" ' '$tainted ' ...
 '"<div onmouseover=\\"x=\\"". $tainted ."\\"\\>"'
 '"<".  $tainted ." href= \\"/bob\\" />" '
 '"<div id=\\"". $tainted ."\\">content</div>" ']


In [92]:
for sink in sink_list:
    if sink[0] == '\"' and sink[-1] == '\"':
        sink = sink[1:-1]
#         print(sink)

In [80]:
with open('/Users/devin/Desktop/sink.txt', mode='w', encoding='utf8') as f:
    for sink in sink_list:
        f.write(sink + '\n')

# 下面是模型

In [93]:
y = np.array(data_pd['label'].tolist())

print(sink_list.shape)
print(y.shape)

(10080,)
(10080,)


In [96]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

CV = CountVectorizer(ngram_range=(3,3), decode_error='ignore',
                    token_pattern=r'\b\w+\b', min_df=1, max_df=1.0)
X = CV.fit_transform(sink_list).toarray()

Tfid = TfidfTransformer(smooth_idf=False)
X_tfid = np.array(Tfid.fit_transform(X).toarray())
print(X_tfid.shape)

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfid, y, random_state=None, test_size = 0.3, stratify=y)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
print(X_train_std.shape)
print(X_test_std[:5])
print(y_train[:5])

(10080, 13)
(7056, 13)
[[-0.40089553 -0.22465298 -0.40706628 -0.32337451 -0.22845583 -0.40706628
  -0.32337451 -0.22465298 -0.22465298 -0.40706628 -0.22465298 -0.22255664
  -0.22500083]
 [ 2.49441546 -0.22465298 -0.40706628 -0.32337451 -0.22845583 -0.40706628
  -0.32337451 -0.22465298 -0.22465298 -0.40706628 -0.22465298 -0.22255664
  -0.22500083]
 [ 2.49441546 -0.22465298 -0.40706628 -0.32337451 -0.22845583 -0.40706628
  -0.32337451 -0.22465298 -0.22465298 -0.40706628 -0.22465298 -0.22255664
  -0.22500083]
 [-0.40089553  4.45130984 -0.40706628 -0.32337451 -0.22845583 -0.40706628
  -0.32337451  4.45130984  4.45130984 -0.40706628  4.45130984 -0.22255664
  -0.22500083]
 [-0.40089553  4.45130984 -0.40706628 -0.32337451 -0.22845583 -0.40706628
  -0.32337451  4.45130984  4.45130984 -0.40706628  4.45130984 -0.22255664
  -0.22500083]]
[0 1 1 1 0]


In [97]:
from sklearn import svm
from sklearn import metrics

svc = svm.SVC(kernel='linear', C=1.0, gamma='auto', probability=True, random_state=None).fit(X_train_std, y_train)  # 线性核
# rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=1.0).fit(X_train, y_train)  # 径向基核
# poly_svc = svm.SVC(kernel='poly', degree=3, C=1.0, gamma='auto').fit(X_train, y_train)  # 多项式核
print('Finished training.\n');

predict_target = svc.predict(X_test_std)
print(metrics.classification_report(y_test, predict_target, 
                                    target_names=['Good sanitize', 'Bad sanitize']))

Finished training.

               precision    recall  f1-score   support

Good sanitize       0.70      0.81      0.75      1718
 Bad sanitize       0.68      0.53      0.60      1306

     accuracy                           0.69      3024
    macro avg       0.69      0.67      0.67      3024
 weighted avg       0.69      0.69      0.69      3024

